In [7]:
days = 20*5

DemandA2 = [3000 3000 2000 2200 2200 0]

Dfactor = 60;
Efactor = 120;
A21factor = 5;

InvA2 = zeros(days+50,1)
InvD = zeros(days+50,1)
InvE = zeros(days+50,1)
InvA21 = zeros(days+50,1)

InvA21[1:end] = 10;

cycleA2 = 1285

t=1;
demandMonth = 1
nOrders=0
noOrder = true
while t < days
    if t >= (demandMonth*20)
        InvA2[t:end] -= DemandA2[demandMonth]
        demandMonth += 1
    end
    if InvA2[t] > DemandA2[demandMonth]
        keepProduction = false
    else
        keepProduction = true
    end
    Dleft = (DemandA2[demandMonth] - InvA2[t]) * Dfactor
    Eleft = (DemandA2[demandMonth] - InvA2[t]) * Efactor
    A21left = (DemandA2[demandMonth] - InvA2[t]) * A21factor
    if ((InvD[t] < Dleft) || (InvE[t] < Eleft) || (InvA21[t] < A21left)) && noOrder && keepProduction
        nOrders+=1
        tLast = t
        tAdd = round(Int64,2*randn(1)[1] + 5)
        if tAdd < 0
            t+=0
        else
            t+=tAdd
        end
        
        InvD[tLast:t] = InvD[tLast]
        InvE[tLast:t] = InvE[tLast]
        InvA21[tLast:t] = InvA21[tLast]
        InvA2[tLast:t] = InvA2[tLast]
        
        InvD[t:end] += max(DemandA2[demandMonth]* Dfactor, cycleA2 * Dfactor * 7)
        InvE[t:end] += max(DemandA2[demandMonth]* Efactor, cycleA2 * Efactor * 7)
        InvA21[t:end] += max(DemandA2[demandMonth]* A21factor, cycleA2 * A21factor * 7)
    else
        if InvA2[t] > DemandA2[demandMonth]
            keepProduction = false
        else
            keepProduction = true
        end
        while ((InvD[t]-Dfactor)>0) && ((InvE[t]-Efactor)>0) && ((InvA21[t]-A21factor)>0) && ((t+1)<= days) && keepProduction          
            
            prod = round(Int64, 200*randn(1)[1] + cycleA2/2)
            nD = div(InvD[t], Dfactor)
            nE = div(InvE[t], Efactor)
            nA21 = div(InvA21[t], A21factor)
            
            nA2 = min(nD, nE, nA21, prod)
            
            InvA2[t:end] += nA2
            InvD[t:end] -= nA2*Dfactor
            InvE[t:end] -= nA2*Efactor
            InvA21[t:end] -= nA2*A21factor            
            defect = rand()/10
            
            InvA2[t:end] -= round(Int64, InvA2[t+1]*defect)
            if InvA2[t] > DemandA2[demandMonth]
                break
            end
            t+=1
            if t >= (demandMonth*20)
                InvA2[t:end] -= DemandA2[demandMonth]
                demandMonth += 1
            end
            Dleft = (DemandA2[demandMonth] - InvA2[t]) * Dfactor
            Eleft = (DemandA2[demandMonth] - InvA2[t]) * Efactor
            A21left = (DemandA2[demandMonth] - InvA2[t]) * A21factor
            if ((InvD[t] < Dleft) || (InvE[t] < Eleft) || (InvA21[t] < A21left)) && noOrder
                noOrder = false
                nOrders += 1
                tLast = t
                tAdd = round(Int64,2*randn(1)[1] + 5)
                if tAdd < 0
                    tOrder=t
                else
                    tOrder=tAdd+t
                end

                InvD[tLast:tOrder] = InvD[tLast]
                InvE[tLast:tOrder] = InvE[tLast]
                InvA21[tLast:tOrder] = InvA21[tLast]
                InvA2[tLast:tOrder] = InvA2[tLast]

                #InvD[tOrder:end] += max(Dleft* Dfactor, cycleA2 * Dfactor * 7)
                #InvE[tOrder:end] += max(Eleft* Efactor, cycleA2 * Efactor * 7)
                #InvA21[tOrder:end] += max(A21left* A21factor, cycleA2 * A21factor * 7)
            end
            if !noOrder
                if !(((InvD[t]-Dfactor)>0) && ((InvE[t]-Efactor)>0) && ((InvA21[t]-A21factor)>0) && ((t+1)<= days) && keepProduction)
                    noOrder = true
                    t = tOrder
                    InvD[tOrder:end] += max(Dleft* Dfactor, cycleA2 * Dfactor * 7)
                    InvE[tOrder:end] += max(Eleft* Efactor, cycleA2 * Efactor * 7)
                    InvA21[tOrder:end] += max(A21left* A21factor, cycleA2 * A21factor * 7)
                end
            end
            
        end
    end
    t+=1
end




In [8]:
using Plots
plot(1:(days+50), InvD/Dfactor, label="D scaled", title="Inventory over 5 months")
plot!(1:(days+50), InvE/Efactor, label = "E scaled")
plot!(1:(days+50), InvA21/A21factor, label="A21 scaled")
plot!(1:(days+50), InvA2, label="A2 actual")
using Images
using FileIO
savefig("a2invall.png")

In [9]:
plot(1:(days+50), InvA2, label="A2", title="Inventory of A2")
using Images
using FileIO
savefig("a2inv.png")